# Data Modeling with Apache Cassandra

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance on local machine 
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
except Exception as e:
    print(e)
    
# Connect a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity_cass WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity_cass')

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [61]:
q1 = """select artist, song, length from music_app_history where sessionId = 338 and itemInSession = 4;"""

create_query = """CREATE TABLE IF NOT EXISTS music_app_history 
                (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"""
drop_query = 'DROP TABLE IF EXISTS music_app_history'

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)

In [64]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [65]:
try:
    rows = session.execute(q1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [66]:
q2 = """select artist, song, firstname, lastname from music_history where userid = 10 and sessionid = 182;"""

create_query2 = """CREATE TABLE IF NOT EXISTS music_history (userid int, sessionid int, itemInSession int, 
                    artist VARCHAR, song VARCHAR, firstname VARCHAR, lastname VARCHAR, 
                    PRIMARY KEY (userid, sessionid, itemInSession))"""
drop_query2 = 'DROP TABLE IF EXISTS music_history'

try:
    session.execute(drop_query2)
except Exception as e:
    print(e)    

try:
    session.execute(create_query2)
except Exception as e:
    print(e)  

In [67]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_history (userid, sessionid, itemInSession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [68]:
try:
    rows = session.execute(q2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' 

In [74]:
q3 = """select firstname, lastname from musicapp_history where song = 'All Hands Against His Own';"""

create_query3 = """CREATE TABLE IF NOT EXISTS musicapp_history (song VARCHAR, firstname VARCHAR, lastname VARCHAR, 
                    PRIMARY KEY (song, firstname, lastname))"""
drop_query3 = 'DROP TABLE IF EXISTS musicapp_history'

try:
    session.execute(drop_query3)
except Exception as e:
    print(e)    

try:
    session.execute(create_query3)
except Exception as e:
    print(e)  

In [75]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO musicapp_history (song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

In [76]:
try:
    rows = session.execute(q3)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [80]:
## Drop the table before closing out the sessions
try:
    session.execute(drop_query)
    session.execute(drop_query2)
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [81]:
session.shutdown()
cluster.shutdown()